# Analyse et Modélisation des Données de Facturation d'Eau

## Objectifs du Projet
- Analyser les données de facturation d'eau
- Prédire les catégories de clients
- Prédire les résiliations
- Prédire les montants de facturation
- Utiliser des techniques robustes pour éviter le surapprentissage

## 1. Import des Bibliothèques

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import glob
from pathlib import Path

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score, 
    mean_squared_error, r2_score, mean_absolute_error
)

# Modèles de Classification
from sklearn.ensemble import (
    RandomForestClassifier, AdaBoostClassifier, 
    GradientBoostingClassifier, BaggingClassifier
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# Modèles de Régression
from sklearn.ensemble import (
    RandomForestRegressor, AdaBoostRegressor, 
    GradientBoostingRegressor, BaggingRegressor
)
from sklearn.linear_model import Ridge, Lasso, ElasticNet

# Gestion du déséquilibre
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("Bibliothèques importées avec succès")

## 2. Chargement des Données

In [ ]:
# Définir les noms de colonnes
colonnes = [
    'DR', 'CEN', 'POLICE', 'O', 'P', 'ENR', 'MM', 'AAAA', 'DATE_FACT', 
    'DIAM', 'CUBCONS', 'CUBFAC', 'FORFAIT', 'SOCIAL', 'DOMEST', 'NORMAL', 
    'INDUST', 'ADMINI', 'MONT_SOD', 'MONT_TVA', 'MONT_FDE', 'MONT_FNE', 
    'MONT_ASS_TTC', 'MONT_FRAIS_CPT', 'MONT_TTC', 'DATE_ABON', 'DATE_RESIL', 
    'TOURNEE', 'DATE_REGLT', 'AAENC', 'MMENC', 'RESILIE', 'CATEGORIE', 
    'NOUVEAU', 'DATE_REGLT_ENC'
]

# Charger tous les fichiers .txt du dossier datasets
chemin_datasets = 'datasets/*.txt'
fichiers = glob.glob(chemin_datasets)

print(f"Nombre de fichiers trouvés: {len(fichiers)}")
print("Fichiers:", fichiers)

# Liste pour stocker les dataframes
liste_df = []

# Charger chaque fichier
for fichier in fichiers:
    try:
        df_temp = pd.read_csv(
            fichier, 
            sep=',',
            names=colonnes,
            encoding='utf-8',
            low_memory=False
        )
        liste_df.append(df_temp)
        print(f"Chargé: {fichier} - {len(df_temp)} lignes")
    except Exception as e:
        print(f"Erreur lors du chargement de {fichier}: {e}")

# Concaténer tous les dataframes
if liste_df:
    df = pd.concat(liste_df, ignore_index=True)
    print(f"\nDataframe final: {df.shape[0]} lignes, {df.shape[1]} colonnes")
else:
    print("Aucune donnée chargée")

## 3. Exploration des Données

In [ ]:
# Aperçu des premières lignes
print("Aperçu des données:")
print(df.head(10))

In [ ]:
# Informations sur les données
print("Informations sur le dataset:")
print(df.info())

In [ ]:
# Statistiques descriptives
print("Statistiques descriptives:")
print(df.describe())

In [ ]:
# Vérifier les valeurs manquantes
print("Valeurs manquantes par colonne:")
valeurs_manquantes = df.isnull().sum()
pourcentage_manquant = (valeurs_manquantes / len(df)) * 100
resume_manquant = pd.DataFrame({
    'Nombre_Manquant': valeurs_manquantes,
    'Pourcentage': pourcentage_manquant
})
print(resume_manquant[resume_manquant['Nombre_Manquant'] > 0].sort_values('Pourcentage', ascending=False))

In [ ]:
# Distribution des catégories
print("Distribution des catégories de clients:")
print(df['CATEGORIE'].value_counts())

print("\nDistribution des résiliations:")
print(df['RESILIE'].value_counts())

In [ ]:
# Visualisation de la distribution des catégories
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Distribution des catégories
df['CATEGORIE'].value_counts().plot(kind='bar', ax=axes[0], color='skyblue')
axes[0].set_title('Distribution des Catégories de Clients', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Catégorie')
axes[0].set_ylabel('Nombre de factures')
axes[0].tick_params(axis='x', rotation=45)

# Distribution des résiliations
df['RESILIE'].value_counts().plot(kind='bar', ax=axes[1], color='coral')
axes[1].set_title('Distribution des Résiliations', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Résilié (1=Oui, 0=Non)')
axes[1].set_ylabel('Nombre de factures')

plt.tight_layout()
plt.show()

## 4. Prétraitement des Données

In [ ]:
# Créer une copie pour le traitement
df_processed = df.copy()

# Convertir les colonnes de dates
colonnes_dates = ['DATE_FACT', 'DATE_ABON', 'DATE_RESIL', 'DATE_REGLT', 'DATE_REGLT_ENC']

for col in colonnes_dates:
    try:
        df_processed[col] = pd.to_datetime(df_processed[col], errors='coerce')
    except Exception as e:
        print(f"Erreur lors de la conversion de {col}: {e}")

print("Colonnes de dates converties")

In [ ]:
# Créer des features temporelles
if pd.api.types.is_datetime64_any_dtype(df_processed['DATE_FACT']):
    df_processed['ANNEE_FACT'] = df_processed['DATE_FACT'].dt.year
    df_processed['MOIS_FACT'] = df_processed['DATE_FACT'].dt.month
    df_processed['TRIMESTRE_FACT'] = df_processed['DATE_FACT'].dt.quarter
    
# Calculer le délai de paiement (en jours)
if pd.api.types.is_datetime64_any_dtype(df_processed['DATE_REGLT']) and \
   pd.api.types.is_datetime64_any_dtype(df_processed['DATE_FACT']):
    df_processed['DELAI_PAIEMENT'] = (
        df_processed['DATE_REGLT'] - df_processed['DATE_FACT']
    ).dt.days
    
# Calculer la durée d'abonnement (en jours)
if pd.api.types.is_datetime64_any_dtype(df_processed['DATE_FACT']) and \
   pd.api.types.is_datetime64_any_dtype(df_processed['DATE_ABON']):
    df_processed['DUREE_ABONNEMENT'] = (
        df_processed['DATE_FACT'] - df_processed['DATE_ABON']
    ).dt.days

print("Features temporelles créées")

In [ ]:
# Créer des features supplémentaires
# Ratio consommation/facturation
df_processed['RATIO_CONS_FAC'] = df_processed['CUBCONS'] / (df_processed['CUBFAC'] + 1)

# Prix unitaire
df_processed['PRIX_UNITAIRE'] = df_processed['MONT_TTC'] / (df_processed['CUBFAC'] + 1)

# Somme des consommations par type
df_processed['TOTAL_CONS_TYPE'] = (
    df_processed['SOCIAL'] + df_processed['DOMEST'] + 
    df_processed['NORMAL'] + df_processed['INDUST'] + df_processed['ADMINI']
)

# Type dominant de consommation
df_processed['TYPE_DOMINANT'] = df_processed[
    ['SOCIAL', 'DOMEST', 'NORMAL', 'INDUST', 'ADMINI']
].idxmax(axis=1)

print("Features supplémentaires créées")

In [ ]:
# Gérer les valeurs infinies et aberrantes
df_processed.replace([np.inf, -np.inf], np.nan, inplace=True)

# Remplir les valeurs manquantes numériques par la médiane
colonnes_numeriques = df_processed.select_dtypes(include=[np.number]).columns

for col in colonnes_numeriques:
    if df_processed[col].isnull().sum() > 0:
        mediane = df_processed[col].median()
        df_processed[col].fillna(mediane, inplace=True)
        
print("Valeurs manquantes traitées")

In [ ]:
# Encoder les variables catégorielles
le_categorie = LabelEncoder()
le_type_dominant = LabelEncoder()
le_mmenc = LabelEncoder()

df_processed['CATEGORIE_ENCODED'] = le_categorie.fit_transform(df_processed['CATEGORIE'].astype(str))
df_processed['TYPE_DOMINANT_ENCODED'] = le_type_dominant.fit_transform(df_processed['TYPE_DOMINANT'].astype(str))
df_processed['MMENC_ENCODED'] = le_mmenc.fit_transform(df_processed['MMENC'].astype(str))

print("Variables catégorielles encodées")
print(f"Catégories: {list(le_categorie.classes_)}")

## 5. Analyse Exploratoire Visuelle

In [ ]:
# Distribution de la consommation
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Consommation par catégorie
df_processed.boxplot(column='CUBCONS', by='CATEGORIE', ax=axes[0, 0])
axes[0, 0].set_title('Distribution de la Consommation par Catégorie', fontweight='bold')
axes[0, 0].set_xlabel('Catégorie')
axes[0, 0].set_ylabel('Consommation (m3)')

# Montant TTC par catégorie
df_processed.boxplot(column='MONT_TTC', by='CATEGORIE', ax=axes[0, 1])
axes[0, 1].set_title('Distribution du Montant TTC par Catégorie', fontweight='bold')
axes[0, 1].set_xlabel('Catégorie')
axes[0, 1].set_ylabel('Montant TTC')

# Consommation par type dominant
df_processed.boxplot(column='CUBCONS', by='TYPE_DOMINANT', ax=axes[1, 0])
axes[1, 0].set_title('Distribution de la Consommation par Type Dominant', fontweight='bold')
axes[1, 0].set_xlabel('Type Dominant')
axes[1, 0].set_ylabel('Consommation (m3)')
axes[1, 0].tick_params(axis='x', rotation=45)

# Délai de paiement par catégorie
if 'DELAI_PAIEMENT' in df_processed.columns:
    df_processed.boxplot(column='DELAI_PAIEMENT', by='CATEGORIE', ax=axes[1, 1])
    axes[1, 1].set_title('Distribution du Délai de Paiement par Catégorie', fontweight='bold')
    axes[1, 1].set_xlabel('Catégorie')
    axes[1, 1].set_ylabel('Délai de Paiement (jours)')

plt.tight_layout()
plt.show()

In [ ]:
# Matrice de corrélation des variables numériques principales
colonnes_correlation = [
    'CUBCONS', 'CUBFAC', 'SOCIAL', 'DOMEST', 'NORMAL', 'INDUST', 'ADMINI',
    'MONT_SOD', 'MONT_TVA', 'MONT_TTC', 'DELAI_PAIEMENT', 'PRIX_UNITAIRE'
]

colonnes_disponibles = [col for col in colonnes_correlation if col in df_processed.columns]

plt.figure(figsize=(14, 10))
matrice_corr = df_processed[colonnes_disponibles].corr()
sns.heatmap(matrice_corr, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Matrice de Corrélation des Variables Principales', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 6. Préparation des Données pour la Modélisation

In [ ]:
# Sélectionner les features pour la modélisation
features_base = [
    'DR', 'CEN', 'POLICE', 'O', 'P', 'DIAM', 'CUBCONS', 'CUBFAC', 
    'FORFAIT', 'SOCIAL', 'DOMEST', 'NORMAL', 'INDUST', 'ADMINI',
    'MONT_SOD', 'MONT_TVA', 'MONT_FDE', 'MONT_FNE', 
    'TOURNEE', 'AAAA', 'MM'
]

features_engineered = [
    'RATIO_CONS_FAC', 'PRIX_UNITAIRE', 'TOTAL_CONS_TYPE',
    'TYPE_DOMINANT_ENCODED', 'MMENC_ENCODED'
]

if 'DELAI_PAIEMENT' in df_processed.columns:
    features_engineered.append('DELAI_PAIEMENT')
if 'DUREE_ABONNEMENT' in df_processed.columns:
    features_engineered.append('DUREE_ABONNEMENT')
if 'ANNEE_FACT' in df_processed.columns:
    features_engineered.append('ANNEE_FACT')
if 'MOIS_FACT' in df_processed.columns:
    features_engineered.append('MOIS_FACT')
if 'TRIMESTRE_FACT' in df_processed.columns:
    features_engineered.append('TRIMESTRE_FACT')

# Combiner toutes les features
all_features = features_base + features_engineered

# Vérifier que toutes les features existent
features_finales = [f for f in all_features if f in df_processed.columns]

print(f"Nombre de features sélectionnées: {len(features_finales)}")
print(f"Features: {features_finales}")

## 7. Modélisation - Prédiction de la Catégorie de Client

In [ ]:
# Préparer les données pour la classification de catégorie
X_categorie = df_processed[features_finales].copy()
y_categorie = df_processed['CATEGORIE_ENCODED'].copy()

# Supprimer les lignes avec des valeurs manquantes
mask = ~(X_categorie.isnull().any(axis=1) | y_categorie.isnull())
X_categorie = X_categorie[mask]
y_categorie = y_categorie[mask]

print(f"Taille du dataset pour la classification: {X_categorie.shape}")
print(f"Distribution des classes:")
print(y_categorie.value_counts())

In [ ]:
# Split train/test stratifié
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(
    X_categorie, y_categorie, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_categorie
)

print(f"Taille train: {X_train_cat.shape}")
print(f"Taille test: {X_test_cat.shape}")

In [ ]:
# Normalisation des features
scaler_cat = StandardScaler()
X_train_cat_scaled = scaler_cat.fit_transform(X_train_cat)
X_test_cat_scaled = scaler_cat.transform(X_test_cat)

print("Features normalisées")

In [ ]:
# Définir les modèles de classification avec techniques anti-surapprentissage
modeles_classification = {
    'Random Forest': RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        min_samples_split=20,
        min_samples_leaf=10,
        random_state=42,
        n_jobs=-1
    ),
    'AdaBoost': AdaBoostClassifier(
        estimator=DecisionTreeClassifier(max_depth=3),
        n_estimators=100,
        learning_rate=0.5,
        random_state=42
    ),
    'Gradient Boosting': GradientBoostingClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        min_samples_split=20,
        min_samples_leaf=10,
        subsample=0.8,
        random_state=42
    ),
    'Bagging': BaggingClassifier(
        estimator=DecisionTreeClassifier(max_depth=10),
        n_estimators=50,
        max_samples=0.8,
        max_features=0.8,
        random_state=42,
        n_jobs=-1
    ),
    'Logistic Regression (L2)': LogisticRegression(
        penalty='l2',
        C=1.0,
        max_iter=1000,
        random_state=42,
        n_jobs=-1
    )
}

print("Modèles de classification définis")

In [ ]:
# Entraîner et évaluer chaque modèle avec validation croisée
resultats_classification = {}
cv_scores = {}

print("Entraînement des modèles de classification...\n")

for nom, modele in modeles_classification.items():
    print(f"Entraînement du modèle: {nom}")
    
    # Validation croisée stratifiée
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_score = cross_val_score(
        modele, 
        X_train_cat_scaled, 
        y_train_cat, 
        cv=skf, 
        scoring='accuracy'
    )
    
    # Entraîner le modèle
    modele.fit(X_train_cat_scaled, y_train_cat)
    
    # Prédictions
    y_pred_train = modele.predict(X_train_cat_scaled)
    y_pred_test = modele.predict(X_test_cat_scaled)
    
    # Scores
    train_score = accuracy_score(y_train_cat, y_pred_train)
    test_score = accuracy_score(y_test_cat, y_pred_test)
    
    # Stocker les résultats
    resultats_classification[nom] = {
        'modele': modele,
        'train_score': train_score,
        'test_score': test_score,
        'y_pred_test': y_pred_test
    }
    
    cv_scores[nom] = cv_score
    
    print(f"Score CV (mean ± std): {cv_score.mean():.4f} ± {cv_score.std():.4f}")
    print(f"Score Train: {train_score:.4f}")
    print(f"Score Test: {test_score:.4f}")
    print(f"Différence Train-Test: {abs(train_score - test_score):.4f}")
    print("-" * 60)
    print()

In [ ]:
# Comparer les performances des modèles
comparaison_df = pd.DataFrame([
    {
        'Modèle': nom,
        'Score CV (mean)': cv_scores[nom].mean(),
        'Score CV (std)': cv_scores[nom].std(),
        'Score Train': res['train_score'],
        'Score Test': res['test_score'],
        'Surapprentissage': abs(res['train_score'] - res['test_score'])
    }
    for nom, res in resultats_classification.items()
]).sort_values('Score Test', ascending=False)

print("Comparaison des modèles de classification:")
print(comparaison_df.to_string(index=False))

In [ ]:
# Visualiser la comparaison des modèles
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scores de test
comparaison_df.plot(x='Modèle', y='Score Test', kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Scores de Test par Modèle', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Accuracy')
axes[0].set_ylim([0, 1])
axes[0].tick_params(axis='x', rotation=45)
axes[0].axhline(y=comparaison_df['Score Test'].max(), color='r', linestyle='--', alpha=0.5)

# Surapprentissage
comparaison_df.plot(x='Modèle', y='Surapprentissage', kind='bar', ax=axes[1], color='coral')
axes[1].set_title('Niveau de Surapprentissage par Modèle', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Différence Train-Test')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Sélectionner le meilleur modèle
meilleur_modele_nom = comparaison_df.iloc[0]['Modèle']
meilleur_modele_cat = resultats_classification[meilleur_modele_nom]['modele']

print(f"Meilleur modèle: {meilleur_modele_nom}")
print(f"Score Test: {comparaison_df.iloc[0]['Score Test']:.4f}")

# Rapport de classification détaillé
y_pred_meilleur = resultats_classification[meilleur_modele_nom]['y_pred_test']

print("\nRapport de classification:")
print(classification_report(
    y_test_cat, 
    y_pred_meilleur, 
    target_names=[str(c) for c in le_categorie.classes_]
))

In [ ]:
# Matrice de confusion
cm = confusion_matrix(y_test_cat, y_pred_meilleur)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=le_categorie.classes_,
            yticklabels=le_categorie.classes_)
plt.title(f'Matrice de Confusion - {meilleur_modele_nom}', fontsize=14, fontweight='bold')
plt.ylabel('Vraie Catégorie')
plt.xlabel('Catégorie Prédite')
plt.tight_layout()
plt.show()

In [ ]:
# Feature importance pour les modèles basés sur les arbres
if hasattr(meilleur_modele_cat, 'feature_importances_'):
    importances = meilleur_modele_cat.feature_importances_
    indices = np.argsort(importances)[::-1][:20]
    
    plt.figure(figsize=(12, 8))
    plt.barh(range(len(indices)), importances[indices], color='teal')
    plt.yticks(range(len(indices)), [features_finales[i] for i in indices])
    plt.xlabel('Importance')
    plt.title(f'Top 20 Features Importantes - {meilleur_modele_nom}', 
              fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

## 8. Modélisation - Prédiction des Résiliations

In [ ]:
# Préparer les données pour la prédiction de résiliation
X_resil = df_processed[features_finales].copy()
y_resil = df_processed['RESILIE'].copy()

# Supprimer les lignes avec des valeurs manquantes
mask = ~(X_resil.isnull().any(axis=1) | y_resil.isnull())
X_resil = X_resil[mask]
y_resil = y_resil[mask]

print(f"Taille du dataset pour la prédiction de résiliation: {X_resil.shape}")
print(f"Distribution des classes:")
print(y_resil.value_counts())
print(f"Pourcentage de résiliations: {(y_resil.sum() / len(y_resil)) * 100:.2f}%")

In [ ]:
# Split train/test stratifié
X_train_resil, X_test_resil, y_train_resil, y_test_resil = train_test_split(
    X_resil, y_resil, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_resil
)

# Normalisation
scaler_resil = StandardScaler()
X_train_resil_scaled = scaler_resil.fit_transform(X_train_resil)
X_test_resil_scaled = scaler_resil.transform(X_test_resil)

print(f"Taille train: {X_train_resil.shape}")
print(f"Taille test: {X_test_resil.shape}")

In [ ]:
# Gérer le déséquilibre de classes avec SMOTE
print("Application de SMOTE pour équilibrer les classes...")

smote = SMOTE(random_state=42)
X_train_resil_balanced, y_train_resil_balanced = smote.fit_resample(
    X_train_resil_scaled, 
    y_train_resil
)

print(f"Taille après SMOTE: {X_train_resil_balanced.shape}")
print(f"Distribution après SMOTE:")
print(pd.Series(y_train_resil_balanced).value_counts())

In [ ]:
# Entraîner et évaluer les modèles pour la résiliation
resultats_resiliation = {}
cv_scores_resil = {}

print("Entraînement des modèles de prédiction de résiliation...\n")

for nom, modele in modeles_classification.items():
    print(f"Entraînement du modèle: {nom}")
    
    # Validation croisée
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_score = cross_val_score(
        modele, 
        X_train_resil_balanced, 
        y_train_resil_balanced, 
        cv=skf, 
        scoring='roc_auc'
    )
    
    # Entraîner le modèle
    modele.fit(X_train_resil_balanced, y_train_resil_balanced)
    
    # Prédictions
    y_pred_train = modele.predict(X_train_resil_balanced)
    y_pred_test = modele.predict(X_test_resil_scaled)
    
    # Scores
    train_score = accuracy_score(y_train_resil_balanced, y_pred_train)
    test_score = accuracy_score(y_test_resil, y_pred_test)
    
    # Stocker les résultats
    resultats_resiliation[nom] = {
        'modele': modele,
        'train_score': train_score,
        'test_score': test_score,
        'y_pred_test': y_pred_test
    }
    
    cv_scores_resil[nom] = cv_score
    
    print(f"Score CV ROC-AUC (mean ± std): {cv_score.mean():.4f} ± {cv_score.std():.4f}")
    print(f"Score Train: {train_score:.4f}")
    print(f"Score Test: {test_score:.4f}")
    print("-" * 60)
    print()

In [ ]:
# Comparer les performances pour la résiliation
comparaison_resil_df = pd.DataFrame([
    {
        'Modèle': nom,
        'Score CV ROC-AUC (mean)': cv_scores_resil[nom].mean(),
        'Score CV ROC-AUC (std)': cv_scores_resil[nom].std(),
        'Score Train': res['train_score'],
        'Score Test': res['test_score']
    }
    for nom, res in resultats_resiliation.items()
]).sort_values('Score Test', ascending=False)

print("Comparaison des modèles pour la prédiction de résiliation:")
print(comparaison_resil_df.to_string(index=False))

In [ ]:
# Meilleur modèle pour la résiliation
meilleur_modele_resil_nom = comparaison_resil_df.iloc[0]['Modèle']
meilleur_modele_resil = resultats_resiliation[meilleur_modele_resil_nom]['modele']
y_pred_resil_meilleur = resultats_resiliation[meilleur_modele_resil_nom]['y_pred_test']

print(f"Meilleur modèle pour la résiliation: {meilleur_modele_resil_nom}")
print(f"Score Test: {comparaison_resil_df.iloc[0]['Score Test']:.4f}")

print("\nRapport de classification:")
print(classification_report(y_test_resil, y_pred_resil_meilleur, 
                          target_names=['Non Résilié', 'Résilié']))

In [ ]:
# Matrice de confusion pour la résiliation
cm_resil = confusion_matrix(y_test_resil, y_pred_resil_meilleur)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_resil, annot=True, fmt='d', cmap='Reds',
            xticklabels=['Non Résilié', 'Résilié'],
            yticklabels=['Non Résilié', 'Résilié'])
plt.title(f'Matrice de Confusion - Résiliation - {meilleur_modele_resil_nom}', 
          fontsize=14, fontweight='bold')
plt.ylabel('Vraie Classe')
plt.xlabel('Classe Prédite')
plt.tight_layout()
plt.show()

## 9. Modélisation - Prédiction du Montant TTC (Régression)

In [ ]:
# Préparer les données pour la régression
features_regression = [f for f in features_finales if f != 'MONT_TTC']

X_reg = df_processed[features_regression].copy()
y_reg = df_processed['MONT_TTC'].copy()

# Supprimer les lignes avec des valeurs manquantes
mask = ~(X_reg.isnull().any(axis=1) | y_reg.isnull())
X_reg = X_reg[mask]
y_reg = y_reg[mask]

print(f"Taille du dataset pour la régression: {X_reg.shape}")
print(f"Statistiques de la variable cible (MONT_TTC):")
print(y_reg.describe())

In [ ]:
# Split train/test
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, 
    test_size=0.2, 
    random_state=42
)

# Normalisation
scaler_reg = StandardScaler()
X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg.transform(X_test_reg)

print(f"Taille train: {X_train_reg.shape}")
print(f"Taille test: {X_test_reg.shape}")

In [ ]:
# Définir les modèles de régression avec régularisation
modeles_regression = {
    'Random Forest': RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        min_samples_split=20,
        min_samples_leaf=10,
        random_state=42,
        n_jobs=-1
    ),
    'AdaBoost': AdaBoostRegressor(
        n_estimators=100,
        learning_rate=0.5,
        random_state=42
    ),
    'Gradient Boosting': GradientBoostingRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        min_samples_split=20,
        min_samples_leaf=10,
        subsample=0.8,
        random_state=42
    ),
    'Ridge': Ridge(
        alpha=1.0,
        random_state=42
    ),
    'Lasso': Lasso(
        alpha=1.0,
        random_state=42,
        max_iter=2000
    ),
    'ElasticNet': ElasticNet(
        alpha=1.0,
        l1_ratio=0.5,
        random_state=42,
        max_iter=2000
    )
}

print("Modèles de régression définis")

In [ ]:
# Entraîner et évaluer les modèles de régression
resultats_regression = {}
cv_scores_reg = {}

print("Entraînement des modèles de régression...\n")

for nom, modele in modeles_regression.items():
    print(f"Entraînement du modèle: {nom}")
    
    # Validation croisée
    cv_score = cross_val_score(
        modele, 
        X_train_reg_scaled, 
        y_train_reg, 
        cv=5, 
        scoring='r2'
    )
    
    # Entraîner le modèle
    modele.fit(X_train_reg_scaled, y_train_reg)
    
    # Prédictions
    y_pred_train = modele.predict(X_train_reg_scaled)
    y_pred_test = modele.predict(X_test_reg_scaled)
    
    # Métriques
    train_r2 = r2_score(y_train_reg, y_pred_train)
    test_r2 = r2_score(y_test_reg, y_pred_test)
    train_rmse = np.sqrt(mean_squared_error(y_train_reg, y_pred_train))
    test_rmse = np.sqrt(mean_squared_error(y_test_reg, y_pred_test))
    test_mae = mean_absolute_error(y_test_reg, y_pred_test)
    
    # Stocker les résultats
    resultats_regression[nom] = {
        'modele': modele,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'test_mae': test_mae,
        'y_pred_test': y_pred_test
    }
    
    cv_scores_reg[nom] = cv_score
    
    print(f"Score CV R2 (mean ± std): {cv_score.mean():.4f} ± {cv_score.std():.4f}")
    print(f"R2 Train: {train_r2:.4f}")
    print(f"R2 Test: {test_r2:.4f}")
    print(f"RMSE Train: {train_rmse:.2f}")
    print(f"RMSE Test: {test_rmse:.2f}")
    print(f"MAE Test: {test_mae:.2f}")
    print("-" * 60)
    print()

In [ ]:
# Comparer les performances des modèles de régression
comparaison_reg_df = pd.DataFrame([
    {
        'Modèle': nom,
        'CV R2 (mean)': cv_scores_reg[nom].mean(),
        'CV R2 (std)': cv_scores_reg[nom].std(),
        'R2 Train': res['train_r2'],
        'R2 Test': res['test_r2'],
        'RMSE Test': res['test_rmse'],
        'MAE Test': res['test_mae']
    }
    for nom, res in resultats_regression.items()
]).sort_values('R2 Test', ascending=False)

print("Comparaison des modèles de régression:")
print(comparaison_reg_df.to_string(index=False))

In [ ]:
# Visualiser les performances des modèles de régression
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# R2 Score
comparaison_reg_df.plot(x='Modèle', y=['R2 Train', 'R2 Test'], 
                       kind='bar', ax=axes[0])
axes[0].set_title('Scores R2 par Modèle', fontsize=14, fontweight='bold')
axes[0].set_ylabel('R2 Score')
axes[0].tick_params(axis='x', rotation=45)
axes[0].legend(['Train', 'Test'])

# RMSE
comparaison_reg_df.plot(x='Modèle', y='RMSE Test', kind='bar', 
                       ax=axes[1], color='coral')
axes[1].set_title('RMSE Test par Modèle', fontsize=14, fontweight='bold')
axes[1].set_ylabel('RMSE')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Meilleur modèle de régression
meilleur_modele_reg_nom = comparaison_reg_df.iloc[0]['Modèle']
meilleur_modele_reg = resultats_regression[meilleur_modele_reg_nom]['modele']
y_pred_reg_meilleur = resultats_regression[meilleur_modele_reg_nom]['y_pred_test']

print(f"Meilleur modèle de régression: {meilleur_modele_reg_nom}")
print(f"R2 Test: {comparaison_reg_df.iloc[0]['R2 Test']:.4f}")
print(f"RMSE Test: {comparaison_reg_df.iloc[0]['RMSE Test']:.2f}")
print(f"MAE Test: {comparaison_reg_df.iloc[0]['MAE Test']:.2f}")

In [ ]:
# Visualiser les prédictions vs valeurs réelles
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scatter plot
axes[0].scatter(y_test_reg, y_pred_reg_meilleur, alpha=0.5, s=10)
axes[0].plot([y_test_reg.min(), y_test_reg.max()], 
            [y_test_reg.min(), y_test_reg.max()], 
            'r--', lw=2)
axes[0].set_xlabel('Valeurs Réelles')
axes[0].set_ylabel('Prédictions')
axes[0].set_title(f'Prédictions vs Valeurs Réelles - {meilleur_modele_reg_nom}', 
                 fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Résidus
residus = y_test_reg - y_pred_reg_meilleur
axes[1].scatter(y_pred_reg_meilleur, residus, alpha=0.5, s=10)
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Prédictions')
axes[1].set_ylabel('Résidus')
axes[1].set_title('Analyse des Résidus', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Optimisation du Meilleur Modèle avec GridSearchCV

In [ ]:
# Optimiser le meilleur modèle de classification avec GridSearchCV
print(f"Optimisation du modèle {meilleur_modele_nom} pour la classification...\n")

# Définir la grille de paramètres selon le modèle
if 'Random Forest' in meilleur_modele_nom:
    param_grid = {
        'n_estimators': [50, 100, 150],
        'max_depth': [8, 10, 12],
        'min_samples_split': [10, 20, 30],
        'min_samples_leaf': [5, 10, 15]
    }
elif 'AdaBoost' in meilleur_modele_nom:
    param_grid = {
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.1, 0.5, 1.0]
    }
elif 'Gradient' in meilleur_modele_nom:
    param_grid = {
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.05, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'subsample': [0.7, 0.8, 0.9]
    }
else:
    param_grid = {}
    print("Pas de grille de paramètres définie pour ce modèle")

if param_grid:
    # GridSearchCV
    grid_search = GridSearchCV(
        estimator=type(meilleur_modele_cat)(),
        param_grid=param_grid,
        cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=42),
        scoring='accuracy',
        n_jobs=-1,
        verbose=1
    )
    
    grid_search.fit(X_train_cat_scaled, y_train_cat)
    
    print(f"\nMeilleurs paramètres: {grid_search.best_params_}")
    print(f"Meilleur score CV: {grid_search.best_score_:.4f}")
    
    # Évaluer sur le test set
    y_pred_optimized = grid_search.predict(X_test_cat_scaled)
    score_optimized = accuracy_score(y_test_cat, y_pred_optimized)
    
    print(f"Score Test après optimisation: {score_optimized:.4f}")
    print(f"Amélioration: {score_optimized - comparaison_df.iloc[0]['Score Test']:.4f}")

## 11. Sauvegarde des Résultats

In [ ]:
# Créer un résumé des résultats
resume_resultats = {
    'Classification - Catégorie': {
        'Meilleur Modèle': meilleur_modele_nom,
        'Score Test': comparaison_df.iloc[0]['Score Test'],
        'Score CV': comparaison_df.iloc[0]['Score CV (mean)'],
        'Surapprentissage': comparaison_df.iloc[0]['Surapprentissage']
    },
    'Classification - Résiliation': {
        'Meilleur Modèle': meilleur_modele_resil_nom,
        'Score Test': comparaison_resil_df.iloc[0]['Score Test'],
        'Score CV ROC-AUC': comparaison_resil_df.iloc[0]['Score CV ROC-AUC (mean)']
    },
    'Régression - Montant TTC': {
        'Meilleur Modèle': meilleur_modele_reg_nom,
        'R2 Test': comparaison_reg_df.iloc[0]['R2 Test'],
        'RMSE Test': comparaison_reg_df.iloc[0]['RMSE Test'],
        'MAE Test': comparaison_reg_df.iloc[0]['MAE Test']
    }
}

print("Résumé des Résultats:")
print("=" * 80)
for tache, resultats in resume_resultats.items():
    print(f"\n{tache}:")
    for metrique, valeur in resultats.items():
        print(f"  {metrique}: {valeur}")

In [ ]:
# Sauvegarder les résultats dans un fichier CSV
import pickle

# Sauvegarder les comparaisons
comparaison_df.to_csv('resultats_classification_categorie.csv', index=False)
comparaison_resil_df.to_csv('resultats_classification_resiliation.csv', index=False)
comparaison_reg_df.to_csv('resultats_regression_montant.csv', index=False)

print("Résultats sauvegardés dans des fichiers CSV")

# Sauvegarder les modèles
with open('meilleur_modele_categorie.pkl', 'wb') as f:
    pickle.dump(meilleur_modele_cat, f)
    
with open('meilleur_modele_resiliation.pkl', 'wb') as f:
    pickle.dump(meilleur_modele_resil, f)
    
with open('meilleur_modele_regression.pkl', 'wb') as f:
    pickle.dump(meilleur_modele_reg, f)

# Sauvegarder les scalers
with open('scaler_categorie.pkl', 'wb') as f:
    pickle.dump(scaler_cat, f)
    
with open('scaler_resiliation.pkl', 'wb') as f:
    pickle.dump(scaler_resil, f)
    
with open('scaler_regression.pkl', 'wb') as f:
    pickle.dump(scaler_reg, f)

print("Modèles et scalers sauvegardés")

## 12. Conclusions et Recommandations

### Conclusions:

1. **Classification des Catégories de Clients:**
   - Le meilleur modèle identifie efficacement les différentes catégories de clients
   - Les features les plus importantes sont liées à la consommation et aux montants facturés
   - Les techniques d'ensemble (Random Forest, Gradient Boosting) performent généralement mieux

2. **Prédiction des Résiliations:**
   - L'utilisation de SMOTE améliore significativement la détection des résiliations
   - Le déséquilibre des classes a été traité efficacement
   - Les modèles peuvent aider à identifier les clients à risque de résiliation

3. **Prédiction des Montants:**
   - Les modèles de régression fournissent des estimations précises des montants TTC
   - La régularisation (Ridge, Lasso, ElasticNet) aide à éviter le surapprentissage
   - Les modèles d'ensemble (Random Forest, Gradient Boosting) capturent bien les relations non-linéaires

### Recommandations:

1. **Pour éviter le surapprentissage:**
   - Utiliser la validation croisée systématiquement
   - Appliquer la régularisation (L1, L2, ou ElasticNet)
   - Limiter la complexité des modèles (max_depth, min_samples_split)
   - Utiliser des techniques d'ensemble (bagging, boosting)

2. **Pour améliorer les performances:**
   - Collecter plus de données sur les résiliations
   - Créer de nouvelles features basées sur le domaine métier
   - Tester d'autres algorithmes (XGBoost, LightGBM)
   - Optimiser les hyperparamètres avec GridSearchCV ou RandomizedSearchCV

3. **Pour la mise en production:**
   - Monitorer les performances des modèles régulièrement
   - Réentraîner les modèles périodiquement avec de nouvelles données
   - Mettre en place des alertes pour détecter la dérive des modèles
   - Documenter les décisions et les résultats pour la traçabilité

### Techniques Anti-Surapprentissage Utilisées:

1. **Validation Croisée Stratifiée:** Pour une évaluation robuste des modèles
2. **Régularisation:** Ridge, Lasso, ElasticNet pour les modèles linéaires
3. **Limitation de la Complexité:** max_depth, min_samples_split, min_samples_leaf
4. **Ensemble Methods:** Random Forest, AdaBoost, Gradient Boosting avec subsample
5. **Early Stopping:** Implicite dans Gradient Boosting
6. **Feature Engineering:** Création de features pertinentes au lieu d'utiliser des features brutes
7. **Traitement du Déséquilibre:** SMOTE pour la classification des résiliations
